## Testing Web Element Detection Model

This notebook loads the pretrained model and saved checkpoints to predict and detect the web page elements. It takes 20 images from the validation dataset and saves the images with bounding boxes at ./results folder

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont


In [2]:
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

/Users/amishra162/Documents/Coursera/work/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/Users/amishra162/Documents/Coursera/work/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ["[Errno 2] The file to load file system plugin from does not exist.: '/Users/amishra162/Documents/Coursera/work/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so'"]
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/Users/amishra162/Documents/Coursera/work/lib/python3.9/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/Users/amishra162/Documents/Coursera/work/lib/python3.9/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ["dlopen(/Users/amishr

In [3]:
# Function to load image as numpy array

def load_image_into_numpy_array(path):
    
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = (Image.open(BytesIO(img_data)))
    (im_width, im_height) = image.size
    
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.float32)

In [4]:
# Function to plot and save the results

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
    
    image_np_with_annotations = image_np.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_annotations,
        boxes,
        classes,
        scores,
        category_index,
        use_normalized_coordinates=True,
        min_score_thresh=0.4)
    
    if image_name:
        plt.imsave(image_name, image_np_with_annotations/255)
    
    else:
        plt.imshow(image_np_with_annotations)

In [5]:
# loading the test data
test_df = pd.read_csv('./data/valid/_annotations.csv')

# test_df['ymin'] = test_df['ymin']/test_df.ymax.max()

# test_df['xmin'] = test_df['xmin']/test_df.xmax.max()

# test_df['ymax'] = test_df['ymax']/test_df.ymax.max()

# test_df['xmax'] = test_df['xmax']/test_df.xmax.max()

In [6]:
print("Number of test images: ",test_df['filename'].unique().shape)

Number of test images:  (482,)


In [7]:
# loading images and converting them to numpy array

img_path = test_df.filename.unique()
test_images_np = [np.expand_dims(load_image_into_numpy_array('./data/valid/'+i), axis=0) for i in img_path]

In [8]:
# Loading the model from checkpoint

pipeline_config = './web_element_detection/new_config/pipeline.config'
checkpoint_path = './web_element_detection/checkpoint/ckpt-1'

configs = config_util.get_configs_from_pipeline_file(pipeline_config)
model_config = configs['model']
model = model_builder.build(model_config=model_config, is_training=True)

checkpoint_path = './web_element_detection/checkpoint/ckpt-1'


checkpoint = tf.train.Checkpoint(
    model=model
)
checkpoint.restore(checkpoint_path)

In [9]:
with open('./web_element_detection/classes.pkl', 'rb') as f:
    category_index = pickle.load(f)

In [10]:
@tf.function
def detect(input_tensor):
    preprocessed_image, shapes = model.preprocess(input_tensor)
    prediction_dict = model.predict(preprocessed_image, shapes)

    detections = model.postprocess(prediction_dict, shapes)
    
    return detections

In [11]:
label_id_offset = 1
results = {'boxes': [], 'scores': []}

for i in range(100,120):
    input_tensor = tf.convert_to_tensor(test_images_np[i])#test_images_np[i], dtype=tf.float32)
    detections = detect(input_tensor)
    plot_detections(
      test_images_np[i][0],
      detections['detection_boxes'][0].numpy(),
      detections['detection_classes'][0].numpy().astype(np.uint32) + label_id_offset,
      detections['detection_scores'][0].numpy(),
      category_index, figsize=(15, 20), image_name="./results/.gif_frame_" + ('%03d' % i) + ".jpg")
    results['boxes'].append(detections['detection_boxes'][0][0].numpy())
    results['scores'].append(detections['detection_scores'][0][0].numpy())

2022-11-29 01:13:33.755246: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
